In [1]:
import sys
sys.path.append('local_modules') # For using module in colab after cloning it

In [1]:
import random
import numpy as np

from torch.utils.data import DataLoader

In [3]:
# Required Dependency for NJSDE
!pip install git+https://github.com/000Justin000/torchdiffeq.git@jj585

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/000Justin000/torchdiffeq.git (to revision jj585) to /tmp/pip-req-build-nsw7gkkw
  Running command git clone --filter=blob:none --quiet https://github.com/000Justin000/torchdiffeq.git /tmp/pip-req-build-nsw7gkkw
  Running command git checkout -b jj585 --track origin/jj585
  Switched to a new branch 'jj585'
  Branch 'jj585' set up to track remote branch 'jj585' from 'origin'.
  Resolved https://github.com/000Justin000/torchdiffeq.git to commit 66d853a05b498d8180e37cff16117b3bf3c91e64
  Preparing metadata (setup.py) ... done
  Created wheel for torchdiffeq: filename=torchdiffeq-0.0.1-py3-none-any.whl size=25548 sha256=edabf7f88b1c5d3e2229d213044073795dd5d61c2785acb1a928bb57576d8e9f
  Stored in directory: /tmp/pip-ephem-wheel-cache-r_whwpsq/wheels/09/f2/fc/fd7e18a76905088c100edec4d254a79ba7e6a490c3f426ddde
Successfully built torchdiffeq


In [4]:
!git clone https://github.com/Prishat/Unified-TPP-Models.git module

Cloning into 'module'...
remote: Enumerating objects: 108, done.
remote: Counting objects: 100% (108/108), done.
remote: Compressing objects: 100% (66/66), done.
remote: Total 108 (delta 49), reused 97 (delta 38), pack-reused 0
Receiving objects: 100% (108/108), 67.03 KiB | 2.58 MiB/s, done.
Resolving deltas: 100% (49/49), done.


#ATM Dataset

In [5]:
from module.datasets import ATMDataset 
from module.config import atm_params

In [6]:
atm_params

{'seq_len': 10, 'batch_size': 4}

In [7]:
train_set = ATMDataset(atm_params, subset='train')
test_set = ATMDataset(atm_params, subset='test')
train_loader = DataLoader(train_set, batch_size=1024, shuffle=True, collate_fn=ATMDataset.to_features)
test_loader = DataLoader(test_set, batch_size=1024, shuffle=False, collate_fn=ATMDataset.to_features)

100%|██████████| 370102/370102 [00:01<00:00, 249647.85it/s]


TOTAL SEQs: 360417
<0.001 = 0.7278214584853152
<0.01 = 0.8774121207208667
<0.1 = 0.9104131204236579
<1 = 0.9390532544378698
<10 = 0.9831806760152387
<100 = 0.9999567696090351


100%|██████████| 182095/182095 [00:00<00:00, 250504.54it/s]

TOTAL SEQs: 177914
<0.001 = 0.7150568634234472
<0.01 = 0.8972010345793314
<0.1 = 0.9240045468773167
<1 = 0.9480623603125704
<10 = 0.9851347863571714
<100 = 0.9999450860227453


In [8]:
from module.models import rmtpp
from module.config import rmtpp_params

In [9]:
rmtpp_params['epochs'] = 2
rmtpp_params

{'model': 'rmtpp',
 'seq_len': 10,
 'emb_dim': 10,
 'hid_dim': 32,
 'mlp_dim': 16,
 'dropout': 0.1,
 'alpha': 0.05,
 'batch_size': 1024,
 'event_class': 7,
 'verbose_step': 350,
 'importance_weight': 'store_true',
 'lr': 0.001,
 'epochs': 2}

In [10]:
model = rmtpp(rmtpp_params)

In [11]:
model.train(train_loader, test_loader)

  0%|          | 0/352 [00:00<?, ?it/s]/content/module/datasets.py:67: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at ../torch/csrc/utils/tensor_new.cpp:245.)
  return torch.FloatTensor(times), torch.LongTensor(events)
100%|██████████| 352/352 [00:08<00:00, 39.49it/s]


time loss:  18.918357224719866
event loss: 1.049259923185621
total loss: 1.9951778505529676


100%|██████████| 174/174 [00:01<00:00, 103.70it/s]


pcnt=7, rcnt=5
epoch 0
time_error: 0.7610035538673401, PRECISION: 0.5341854625222615, RECALL: 0.7067442402520185, F1: 0.6084671807273847


100%|██████████| 352/352 [00:04<00:00, 76.00it/s]


time loss:  4.8633256563118525
event loss: 0.5614005883250918
total loss: 0.8045668807200023


100%|██████████| 174/174 [00:01<00:00, 95.43it/s]


pcnt=7, rcnt=5
epoch 1
time_error: 1.1419694423675537, PRECISION: 0.6531598125438031, RECALL: 0.889971823401609, F1: 0.7533949998842803


#StackOverflow Dataset

In [2]:
from module.datasets import stackOverflow

In [3]:
data = stackOverflow({}, './so/')

In [4]:
train = stackOverflow.process_rmtpp(data.ts[:100])
test = stackOverflow.process_rmtpp(data.ts[100:200])
train_loader = DataLoader(train, batch_size=4, shuffle=True, collate_fn=stackOverflow.to_features)
test_loader = DataLoader(test, batch_size=4, shuffle=False, collate_fn=stackOverflow.to_features)

In [5]:
from module.models import rmtpp
from module.config import rmtpp_params

In [6]:
rmtpp_params['event_class'] = 22
rmtpp_params['epochs'] = 2
rmtpp_params

{'model': 'rmtpp',
 'seq_len': 10,
 'emb_dim': 10,
 'hid_dim': 32,
 'mlp_dim': 16,
 'dropout': 0.1,
 'alpha': 0.05,
 'batch_size': 1024,
 'event_class': 22,
 'verbose_step': 350,
 'importance_weight': 'store_true',
 'lr': 0.001,
 'epochs': 2}

In [7]:
model = rmtpp(rmtpp_params)

In [8]:
model.train(train_loader, test_loader)

  0%|          | 0/25 [00:00<?, ?it/s]/content/module/datasets.py:323: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at ../torch/csrc/utils/tensor_new.cpp:245.)
  return torch.FloatTensor(times), torch.LongTensor(events)
100%|██████████| 25/25 [00:00<00:00, 970.99it/s]


pcnt=11, rcnt=2
epoch 0
time_error: 0.4215855002403259, PRECISION: 0.08897485493230174, RECALL: 0.25, F1: 0.1312410841654779


100%|██████████| 25/25 [00:00<00:00, 952.84it/s]

pcnt=11, rcnt=1
epoch 1
time_error: 0.3346892297267914, PRECISION: 0.09090909090909091, RECALL: 0.47, F1: 0.15235008103727712


#Mimic2 Dataset

In [1]:
from module.datasets import thpDataloader
from module.config import thp_data_params

In [2]:
thp_data_params

{'data': './mimic/fold1/', 'batch_size': 4}

In [3]:
mimic = thpDataloader(thp_data_params)

In [4]:
trainloader, testloader, num_types = mimic.load_data()

[Info] Loading train data...
[Info] Loading dev data...
[Info] Loading test data...


In [5]:
tset, vset = thpDataloader.process_rmtpp(trainloader, testloader, mn_size = 4)

In [6]:
from module.models import rmtpp
from module.config import rmtpp_params

In [7]:
rmtpp_params['event_class'] = num_types+1
rmtpp_params['epochs'] = 2
rmtpp_params

{'model': 'rmtpp',
 'seq_len': 10,
 'emb_dim': 10,
 'hid_dim': 32,
 'mlp_dim': 16,
 'dropout': 0.1,
 'alpha': 0.05,
 'batch_size': 1024,
 'event_class': 76,
 'verbose_step': 350,
 'importance_weight': 'store_true',
 'lr': 0.001,
 'epochs': 2}

In [8]:
model = rmtpp(rmtpp_params)

In [9]:
model.train(tset, vset)

100%|██████████| 15/15 [00:00<00:00, 379.62it/s]


pcnt=8, rcnt=1
epoch 0
time_error: 1.387802004814148, PRECISION: 0.125, RECALL: 0.5263157894736842, F1: 0.20202020202020202


100%|██████████| 15/15 [00:00<00:00, 1046.69it/s]

pcnt=8, rcnt=1
epoch 1
time_error: 1.060810923576355, PRECISION: 0.125, RECALL: 0.5263157894736842, F1: 0.20202020202020202


#PilotElevator Dataset

In [1]:
from module.datasets import nhpDatareader, thpDataloader
from module.config import nhp_dataset_params

In [2]:
nhp_dataset_params

{'PathData': './data/'}

In [3]:
data = nhpDatareader(nhp_dataset_params)

In [4]:
train_data, val_data, total_event_num = data.read_data()

In [5]:
from module.misc import get_dataloader

In [6]:
trainloader = get_dataloader(train_data, batch_size=4, shuffle=True)
testloader = get_dataloader(val_data, batch_size=4, shuffle=False)

In [7]:
tset, vset = thpDataloader.process_rmtpp(trainloader, testloader, mn_size = 4)

In [8]:
from module.models import rmtpp
from module.config import rmtpp_params

In [9]:
rmtpp_params['event_class'] = total_event_num+1
rmtpp_params['epochs'] = 2
rmtpp_params

{'model': 'rmtpp',
 'seq_len': 10,
 'emb_dim': 10,
 'hid_dim': 32,
 'mlp_dim': 16,
 'dropout': 0.1,
 'alpha': 0.05,
 'batch_size': 1024,
 'event_class': 11,
 'verbose_step': 350,
 'importance_weight': 'store_true',
 'lr': 0.001,
 'epochs': 2}

In [10]:
model = rmtpp(rmtpp_params)

In [11]:
model.train(tset, vset)

100%|██████████| 25/25 [00:00<00:00, 953.81it/s]


pcnt=10, rcnt=1
epoch 0
time_error: 19.213621139526367, PRECISION: 0.1, RECALL: 0.51, F1: 0.16721311475409836


100%|██████████| 25/25 [00:00<00:00, 1243.89it/s]

pcnt=10, rcnt=1
epoch 1
time_error: 19.219345092773438, PRECISION: 0.1, RECALL: 0.51, F1: 0.16721311475409836
